<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Francsico Flores &bull; 2025-01
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [16]:
# --- Importación de paquetes necesarios ---

# Manejo de datos
import os
import pandas as pd
import numpy as np

# Modelos y búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

# Evaluación del modelo
from sklearn.metrics import accuracy_score

# Guardado y carga del modelo entrenado
import joblib


---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [18]:
# --- Definir la ruta del archivo CSV ---
dataset_path = "Base/emails.csv"  # Ajustar la ruta si es necesario

# Verificar si el archivo existe antes de cargarlo
if os.path.exists(dataset_path):
    print(f"Archivo encontrado en: {dataset_path}")
else:
    raise FileNotFoundError(f"No se encontró el archivo en {dataset_path}. Verifica la ruta.")

Archivo encontrado en: Base/emails.csv


In [20]:
# --- Cargar el dataset ---
df = pd.read_csv(dataset_path)

# Mostrar las primeras filas para verificar la carga correcta
print("Primeras filas del dataset:")
display(df.head())  # Se usa print en lugar de display para mayor compatibilidad

Primeras filas del dataset:


,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [21]:
# --- Verificar valores nulos en el dataset ---
# Se muestra el número de valores nulos por columna para identificar posibles problemas en los datos
print("Valores nulos por columna:")
print(df.isnull().sum())

Valores nulos por columna:
Email No.     0
the           0
to            0
ect           0
and           0
             ..
military      0
allowing      0
ff            0
dry           0
Prediction    0
Length: 3002, dtype: int64


In [ ]:
# --- Eliminar la columna con los nombres de los correos ---
# Se eliminan datos innecesarios dejando solo las características y la etiqueta de clasificación
df = df.iloc[:, 1:]  

# --- Convertir la etiqueta a formato numérico ---
# Si la columna de etiquetas está en formato texto, se convierte a valores numéricos
if df.iloc[:, -1].dtype == 'object':
    df.iloc[:, -1] = df.iloc[:, -1].map({'spam': 1, 'ham': 0})

# --- Definir variables independientes (X) y dependiente (y) ---
X = df.iloc[:, :-1] # X contiene todas las características excepto la etiqueta
y = df.iloc[:, -1] # y contiene la etiqueta de clasificación (spam/no spam)

# --- Verificar dimensiones de los datos ---
# Se imprime el tamaño de las matrices X e y para confirmar la correcta separación de variables
print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")

Dimensiones de X: (5172, 3000)
Dimensiones de y: (5172,)



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [7]:
# --- Inicializar el modelo Random Forest ---
# Se crea un modelo sin ajuste de hiperparámetros para evaluar su rendimiento base
rf = RandomForestClassifier(random_state=42)

# --- Mostrar los hiperparámetros por defecto ---
# Se imprimen los valores predeterminados de los hiperparámetros del modelo
print("Hiperparámetros por defecto:")
print(rf.get_params())

Hiperparámetros por defecto:
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


**🔍 Los 4 hiperparámetros más importantes en Random Forest**

**`n_estimators` (Número de árboles en el bosque)**
- Define cuántos árboles se entrenarán en el modelo.
- Más árboles suelen mejorar el rendimiento, pero aumentan el tiempo de entrenamiento.
- **Valor por defecto:** 100.

**`max_depth` (Profundidad máxima de los árboles)**
- Controla cuán profundo pueden crecer los árboles.
- Si es `None`, los árboles crecerán hasta que todas las hojas sean puras o contengan menos de `min_samples_split`.
- Valores altos pueden hacer que el modelo sea más complejo y propenso a sobreajuste.

**`min_samples_split` (Mínimo de muestras para dividir un nodo)**
- Determina el número mínimo de muestras que debe tener un nodo para dividirse en dos nodos hijos.
- Un valor bajo da modelos más detallados, mientras que un valor alto los hace más simples.
- **Valor por defecto:** 2.

**`max_features` (Número máximo de características consideradas en cada división)**
- Controla cuántas variables se usan en cada nodo del árbol.
- `'sqrt'` (por defecto) selecciona la raíz cuadrada del número total de características.
- `'log2'` usa el logaritmo en base 2 del número de características.
- También se puede definir como un número entero o como un porcentaje (`float`) de características.


Los hiperparámetros que serán optimizados para encontrar la mejor combinación y mejorar el rendimiento del modelo son: `n_estimators`, `max_depth`, `min_samples_split`, `max_features`, `min_samples_leaf`

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [13]:
# --- Dividir el dataset en entrenamiento y prueba ---
# Se asigna el 80% de los datos para entrenamiento y el 20% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Definir el modelo base ---
# Se utiliza un modelo de Random Forest sin ajuste de hiperparámetros
rf = RandomForestClassifier(random_state=42)

# --- Definir el espacio de búsqueda de hiperparámetros ---
# Se utilizan valores reducidos para optimizar el tiempo de búsqueda
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles en el bosque
    'max_depth': [10, 20, None],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5],  # Número mínimo de muestras para dividir un nodo
    'max_features': ['sqrt', 'log2'],  # Selección de características en cada división
    'min_samples_leaf': [1, 2]  # Número mínimo de muestras en una hoja
}

# --- Configurar la búsqueda de hiperparámetros ---
# Se utiliza GridSearchCV con validación cruzada para encontrar la mejor combinación
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,  # Validación cruzada con 5 particiones
    scoring='accuracy',  # Métrica de evaluación basada en precisión
    n_jobs=-1,  # Uso de todos los núcleos del procesador para acelerar la búsqueda
    verbose=2  # Muestra el progreso del entrenamiento
)

# --- Ejecutar la búsqueda de hiperparámetros ---
# Se entrena el modelo probando todas las combinaciones del grid
grid_search.fit(X_train, y_train)

# --- Mostrar los mejores hiperparámetros encontrados ---
# Se imprimen los valores óptimos seleccionados por GridSearchCV
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# --- Evaluar el modelo optimizado en el conjunto de prueba ---
# Se utiliza el mejor modelo encontrado para realizar predicciones en los datos de prueba
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

# --- Calcular y mostrar la precisión del modelo ---
# Se compara la predicción del modelo con las etiquetas reales del conjunto de prueba
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores hiperparámetros encontrados:
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Precisión en el conjunto de prueba: 0.9749


### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [11]:
# --- Cargar dataset ---
df = pd.read_csv("Base/emails.csv")  # Ajustar la ruta si es necesario

# --- Preparar variables independientes (X) y dependientes (y) ---
X = df.iloc[:, 1:-1].to_numpy().astype(np.float32)  # Convertir a formato NumPy
y = df.iloc[:, -1].to_numpy().astype(np.int32)  # Etiquetas ya están en 0 y 1

# --- Verificar dimensiones ---
print(f"Dimensiones de X: {X.shape}")  # Debe ser (5172, 3000)
print(f"Dimensiones de y: {y.shape}")  # Debe ser (5172,)

# --- Dividir en entrenamiento y prueba ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Definir modelo base ---
rf = RandomForestClassifier(random_state=42, n_jobs=-1)  # Se usa la CPU de forma óptima

# --- Espacio de búsqueda de hiperparámetros ---
param_dist = {
    'n_estimators': [50, 100, 200, 300, 500],  
    'max_depth': [10, 20, 30, 40, None],  
    'min_samples_split': [2, 5, 10, 15, 20],  
    'max_features': ['sqrt', 'log2', None, 0.5, 0.75],  
    'min_samples_leaf': [1, 2, 4, 6, 8]  
}

# --- Configurar búsqueda aleatoria de hiperparámetros ---
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=25,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# --- Entrenar el modelo ---
print("Iniciando entrenamiento con búsqueda de hiperparámetros...")
random_search.fit(X_train, y_train)

# --- Mostrar mejores hiperparámetros ---
print("Mejores hiperparámetros encontrados:")
print(random_search.best_params_)

# --- Evaluar modelo en el conjunto de prueba ---
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")


Dimensiones de X: (5172, 3000)
Dimensiones de y: (5172,)
Iniciando entrenamiento con búsqueda de hiperparámetros...
Fitting 5 folds for each of 25 candidates, totalling 125 fits
Mejores hiperparámetros encontrados:
{'n_estimators': 500, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 30}
Precisión en el conjunto de prueba: 0.9749


### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [17]:
# --- Definir los mejores hiperparámetros encontrados ---
best_params = {
    'n_estimators': 500,
    'min_samples_split': 15,
    'min_samples_leaf': 2,
    'max_features': 'sqrt',
    'max_depth': 30,
    'random_state': 42
}

# --- Inicializar y entrenar el modelo con los mejores hiperparámetros ---
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train, y_train)

# --- Evaluar el modelo optimizado ---
y_pred = best_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión final del modelo reentrenado: {accuracy:.4f}")

# --- Guardar el modelo entrenado ---
model_filename = "modelo_random_forest.pkl"
joblib.dump(best_rf, model_filename)
print(f"Modelo guardado como: {model_filename}")


Precisión final del modelo reentrenado: 0.9749
Modelo guardado como: modelo_random_forest.pkl


### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>